In [1]:
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed

In [48]:
train_one = pd.read_csv('train_with_feats.csv')
train_two = pd.read_csv('train_interaction_raddar.csv')

In [4]:
print ('The train_one file has {} rows and {} columns'.format(train_one.shape[0],train_one.shape[1]))
print ('The train_two file has {} rows and {} columns'.format(train_two.shape[0], train_two.shape[1]))

The train_one file has 4208 rows and 708 columns
The train_two file has 4209 rows and 149 columns


In [5]:
'y' in train_two.columns

True

In [49]:
train_two['y'].isnull().value_counts()

False    4209
Name: y, dtype: int64

In [6]:
test_one = pd.read_csv('test_with_feats.csv')
test_two = pd.read_csv('test_interaction_raddar.csv')

In [51]:
print ('The test_one file has {} rows and {} columns'.format(test_one.shape[0],test_one.shape[1]))
print ('The test_two file has {} rows and {} columns'.format(test_two.shape[0], test_two.shape[1]))

The test_one file has 4209 rows and 707 columns
The test_two file has 4209 rows and 148 columns


In [52]:
train_two = train_two.loc[train_two['y'] < 180]

In [53]:
train_two['y'].isnull().value_counts()

False    4208
Name: y, dtype: int64

In [112]:
print train_one.shape

(4208, 708)


In [113]:
print (train_two.shape)

(4208, 149)


In [119]:
full_train = pd.concat([train_one.reset_index(drop=True),train_two],axis=1)

In [ ]:
df_c = pd.concat([df_a.reset_index(drop=True), df_b], axis=1)

In [120]:
full_train.shape

(4209, 857)

In [116]:
full_train['y'].isnull().value_counts()

False    4208
True        1
Name: y, dtype: int64

In [74]:
full_train.loc[full_train['y'].isnull()]

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X1X3X4X6X8,X1X3X5X6X8,X1X4X5X6X8,X2X3X4X5X6,X2X3X4X5X8,X2X3X4X6X8,X2X3X5X6X8,X2X4X5X6X8,X3X4X5X6X8,y
883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
full_train.describe()

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X1X3X4X6X8,X1X3X5X6X8,X1X4X5X6X8,X2X3X4X5X6,X2X3X4X5X8,X2X3X4X6X8,X2X3X5X6X8,X2X4X5X6X8,X3X4X5X6X8,y
count,4208.000000,4208.0,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,...,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000,4208.000000
mean,0.013308,0.0,0.075095,0.057985,0.428232,0.000475,0.002614,0.007605,0.007842,0.099335,...,778.112167,1815.261407,1718.836502,1047.196293,1438.636644,768.029705,1825.672766,1677.408983,1567.319629,100.630190
std,0.114604,0.0,0.263576,0.233742,0.494881,0.021798,0.051067,0.086882,0.088219,0.299146,...,581.003667,1057.198620,1006.385795,619.114332,839.963312,568.041848,1062.164711,979.572880,909.356900,12.424146
min,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,72.110000
25%,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,245.000000,896.750000,838.750000,499.750000,702.750000,262.750000,901.750000,821.750000,778.750000,90.817500
50%,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,671.000000,1817.500000,1720.500000,1042.000000,1450.000000,654.000000,1829.500000,1676.500000,1572.500000,99.150000
75%,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1254.000000,2726.250000,2590.250000,1576.000000,2163.000000,1209.250000,2746.250000,2530.250000,2347.250000,109.010000
max,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2016.000000,3660.000000,3472.000000,2159.000000,2914.000000,1999.000000,3670.000000,3380.000000,3167.000000,169.910000


In [55]:
full_train.head()

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X1X3X4X6X8,X1X3X5X6X8,X1X4X5X6X8,X2X3X4X5X6,X2X3X4X5X8,X2X3X4X6X8,X2X3X5X6X8,X2X4X5X6X8,X3X4X5X6X8,y
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,130.81
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,88.53
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,76.26
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,80.62
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,78.02


In [21]:
full_test = pd.concat([test_one.reset_index(drop=True), test_two],axis=1)

In [56]:
full_test.head()

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X1X3X4X5X8,X1X3X4X6X8,X1X3X5X6X8,X1X4X5X6X8,X2X3X4X5X6,X2X3X4X5X8,X2X3X4X6X8,X2X3X5X6X8,X2X4X5X6X8,X3X4X5X6X8
0,0,0,0,0,0,0,0,0,0,0,...,2889,2017,3661,3473,2160,2915,2000,3671,3381,3168
1,0,0,0,0,0,0,0,0,0,1,...,2890,1130,3662,3474,2161,2916,2001,3672,3382,3169
2,0,0,0,0,1,0,0,0,0,0,...,2891,1273,3663,3475,2162,2917,1316,3673,3383,3170
3,0,0,0,0,0,0,0,0,0,0,...,2892,1344,3664,3476,2163,2918,2002,3674,3384,3171
4,0,0,0,0,1,0,0,0,0,0,...,2893,1678,3665,3477,2164,2919,1226,3675,3385,3172


In [24]:
col_diff = []
for var in full_train.columns:
    if var not in full_test.columns:
        col_diff.append(var)

In [25]:
print col_diff

['ID', 'y']


In [65]:
y_train = train_two['y'].values

In [69]:
pd.isnull(y_train).any()

False

In [70]:
y_mean = np.mean(y_train)

In [78]:
'ID' in full_test.columns

False

## Joining data in R and using Here

In [146]:
full_train = pd.read_csv("full_train.csv")
full_test = pd.read_csv("full_test.csv")

In [147]:
pd.set_option("display.max_columns",1000)

In [148]:
print full_train.shape
print full_test.shape

(4208, 857)
(4209, 855)


In [149]:
full_train.head()

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X268,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X289,X290,X291,X292,X293,X294,X295,X296,X297,X298,X299,X300,X301,X302,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385,X0a,X0aa,X0ab,X0ac,X0ad,X0ae,X0af,X0ag,X0ai,X0aj,X0ak,X0al,X0am,X0an,X0ao,X0ap,X0aq,X0as,X0at,X0au,X0av,X0aw,X0ax,X0ay,X0az,X0b,X0ba,X0bb,X0bc,X0c,X0d,X0e,X0f,X0g,X0h,X0i,X0j,X0k,X0l,X0m,X0n,X0o,X0p,X0q,X0r,X0s,X0t,X0u,X0v,X0w,X0x,X0y,X0z,X1a,X1aa,X1ab,X1b,X1c,X1d,X1e,X1f,X1g,X1h,X1i,X1j,X1k,X1l,X1m,X1n,X1o,X1p,X1q,X1r,X1s,X1t,X1u,X1v,X1w,X1y,X1z,X2a,X2aa,X2ab,X2ac,X2ad,X2ae,X2af,X2ag,X2ah,X2ai,X2aj,X2ak,X2al,X2am,X2an,X2ao,X2ap,X2aq,X2ar,X2as,X2at,X2au,X2av,X2aw,X2ax,X2ay,X2b,X2c,X2d,X2e,X2f,X2g,X2h,X2i,X2j,X2k,X2l,X2m,X2n,X2o,X2p,X2q,X2r,X2s,X2t,X2u,X2w,X2x,X2y,X2z,X3a,X3b,X3c,X3d,X3e,X3f,X3g,X4a,X4b,X4c,X4d,X5a,X5aa,X5ab,X5ac,X5ad,X5ae,X5af,X5ag,X5ah,X5b,X5c,X5d,X5f,X5g,X5h,X5i,X5j,X5k,X5l,X5m,X5n,X5o,X5p,X5q,X5r,X5s,X5t,X5u,X5v,X5w,X5x,X5y,X5z,X6a,X6b,X6c,X6d,X6e,X6f,X6g,X6h,X6i,X6j,X6k,X6l,X8a,X8b,X8c,X8d,X8e,X8f,X8g,X8h,X8i,X8j,X8k,X8l,X8m,X8n,X8o,X8p,X8q,X8r,X8s,X8t,X8u,X8v,X8w,X8x,X8y,row_sum,LPC1,LPC2,LPC3,LPC4,LPC5,LPC6,LPC7,LPC8,LPC9,LPC10,LPC11,LPC12,LPC13,LPC14,LPC15,LPC16,LPC17,LPC18,LPC19,LPC20,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59,PC60,PC61,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72,PC73,PC74,PC75,PC76,PC77,PC78,PC79,PC80,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90,PC91,PC92,PC93,PC94,hclusters,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,ID,X0,X1,X2,X3,X4,X5,X6,X8,X0X1,X0X2,X0X3,X0X4,X0X5,X0X6,X0X8,X1X2,X1X3,X1X4,X1X5,X1X6,X1X8,X2X3,X2X4,X2X5,X2X6,X2X8,X3X4,X3X5,X3X6,X3X8,X4X5,X4X6,X4X8,X5X6,X5X8,X6X8,X0X1X2,X0X1X3,X0X1X4,X0X1X5,X0X1X6,X0X1X8,X0X2X3,X0X2X4,X0X2X5,X0X2X6,X0X2X8,X0X3X4,X0X3X5,X0X3X6,X0X3X8,X0X4X5,X0X4X6,X0X4X8,X0X5X6,X0X5X8,X0X6X8,X1X2X3,X1X2X4,X1X2X5,X1X2X6,X1X2X8,X1X3X4,X1X3X5,X1X3X6,X1X3X8,X1X4X5,X1X4X6,X1X4X8,X1X5X6,X1X5X8,X1X6X8,X2X3X4,X2X3X5,X2X3X6,X2X3X8,X2X4X5,X2X4X6,X2X4X8,X2X5X6,X2X5X8,X2X6X8,X3X4X5,X3X4X6,X3X4X8,X3X5X6,X3X5X8,X3X6X8,X4X5X6,X4X5X8,X4X6X8,X5X6X8,X0X1X2X3X4,X0X1X2X3X5,X0X1X2X3X6,X0X1X2X3X8,X0X1X2X4X5,X0X1X2X4X6,X0X1X2X4X8,X0X1X2X5X6,X0X1X2X5X8,X0X1X2X6X8,X0X1X3X4X5,X0X1X3X4X6,X0X1X3X4X

In [150]:
y_train = full_train['y']

In [151]:
len(y_train)

4208

In [152]:
y_mean = np.mean(y_train)

In [154]:
y_mean

100.63019011406843

In [155]:
def rsquare(preds, dtrain):
    labels = dtrain.get_label()
    return  'r2', r2_score(labels,preds)

In [156]:
xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

In [132]:
'y' in full_train.columns

False

In [157]:
dtrain = xgb.DMatrix(full_train.drop(['ID','y'], axis=1), y_train)
dtest = xgb.DMatrix(full_test)


In [158]:
len(y_train)

4208

In [144]:
full_train.shape

(4208, 856)

In [159]:
cv = xgb.cv(xgb_params, dtrain, 2000,\
            nfold=5L, feval = rsquare, maximize=True,early_stopping_rounds=40,verbose_eval=20)

[0]	train-r2:0.005492+9.13302e-05	test-r2:0.0050528+0.00018215
[20]	train-r2:0.105788+0.0012707	test-r2:0.102322+0.00134086
[40]	train-r2:0.1899+0.00242448	test-r2:0.18315+0.0027186
[60]	train-r2:0.260599+0.00333296	test-r2:0.250754+0.00401494
[80]	train-r2:0.320045+0.00405742	test-r2:0.307168+0.00546594
[100]	train-r2:0.370048+0.00479537	test-r2:0.353999+0.00700146
[120]	train-r2:0.412259+0.00537243	test-r2:0.393285+0.00839676
[140]	train-r2:0.447882+0.00588014	test-r2:0.426176+0.0097782
[160]	train-r2:0.477955+0.00628698	test-r2:0.45362+0.011
[180]	train-r2:0.50348+0.00664951	test-r2:0.476557+0.0122915
[200]	train-r2:0.52518+0.00690731	test-r2:0.495598+0.013507
[220]	train-r2:0.543592+0.00712392	test-r2:0.511454+0.0147246
[240]	train-r2:0.559366+0.00728464	test-r2:0.524594+0.0159312
[260]	train-r2:0.572844+0.00744013	test-r2:0.535654+0.0170275
[280]	train-r2:0.584447+0.00758365	test-r2:0.544872+0.018226
[300]	train-r2:0.594505+0.00761123	test-r2:0.552437+0.0192631
[320]	train-r2:0.60

In [160]:
num_boost_rounds = 760

In [161]:
model_one = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

In [163]:
y_pred = model_one.predict(dtest)

In [164]:
y_pred[1:10]

array([  92.3926239 ,   79.31860352,   78.69511414,  110.65158081,
         91.81554413,  109.84669495,   93.37440491,  114.2719574 ,
         93.07704926], dtype=float32)

In [165]:
'''Train the stacked models then predict the test data'''

stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)),
    LassoLarsCV()

)

usable_columns = list(set(full_train.columns) - set(['y','ID']))

finaltrainset = full_train[usable_columns].values
finaltestset = full_test.values

stacked_pipeline.fit(finaltrainset, y_train)
results = stacked_pipeline.predict(finaltestset)

In [166]:
usable_columns = list(set(full_train.columns) - set(['y','ID']))

In [168]:
'y' in full_test

False

In [169]:
finaltrainset = full_train[usable_columns].values
finaltestset = full_test.values

In [170]:
stacked_pipeline.fit(finaltrainset, y_train)
results = stacked_pipeline.predict(finaltestset)

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=6.030e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:377: RuntimeWarning: overflow encountered in divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:381: RuntimeWarning: overflow encountered in divide
  g2 = arrayfuncs.min_pos((C + Cov) / (AA + corr_eq_dir + tiny))
/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=5.6

In [172]:
print(r2_score(y_train,stacked_pipeline.predict(finaltrainset)*0.2855 + model_one.predict(dtrain)*0.7145))


0.661366231456


In [173]:
test = pd.read_csv('test.csv')

In [174]:
'''Average the preditionon test data  of both models then save it on a csv file'''

sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['y'] = y_pred*0.75 + results*0.25
sub.to_csv('20thJune/stacked-models.csv', index=False) #-15529.4712

In [175]:
sub.describe()

,ID,y
count,4209.000000,4209.000000
mean,4211.039202,-668.034529
std,2423.078926,1312.967127
min,1.000000,-5084.217717
25%,2115.000000,-901.244850
50%,4202.000000,-310.704851
75%,6310.000000,149.742444
max,8416.000000,2989.805456


### this time lets try without interaction features

In [176]:
train_feat = pd.read_csv("train_with_feats.csv")
test_feat = pd.read_csv("test_with_feats.csv")

In [183]:
train_feat.head()

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X268,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X289,X290,X291,X292,X293,X294,X295,X296,X297,X298,X299,X300,X301,X302,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385,X0a,X0aa,X0ab,X0ac,X0ad,X0ae,X0af,X0ag,X0ai,X0aj,X0ak,X0al,X0am,X0an,X0ao,X0ap,X0aq,X0as,X0at,X0au,X0av,X0aw,X0ax,X0ay,X0az,X0b,X0ba,X0bb,X0bc,X0c,X0d,X0e,X0f,X0g,X0h,X0i,X0j,X0k,X0l,X0m,X0n,X0o,X0p,X0q,X0r,X0s,X0t,X0u,X0v,X0w,X0x,X0y,X0z,X1a,X1aa,X1ab,X1b,X1c,X1d,X1e,X1f,X1g,X1h,X1i,X1j,X1k,X1l,X1m,X1n,X1o,X1p,X1q,X1r,X1s,X1t,X1u,X1v,X1w,X1y,X1z,X2a,X2aa,X2ab,X2ac,X2ad,X2ae,X2af,X2ag,X2ah,X2ai,X2aj,X2ak,X2al,X2am,X2an,X2ao,X2ap,X2aq,X2ar,X2as,X2at,X2au,X2av,X2aw,X2ax,X2ay,X2b,X2c,X2d,X2e,X2f,X2g,X2h,X2i,X2j,X2k,X2l,X2m,X2n,X2o,X2p,X2q,X2r,X2s,X2t,X2u,X2w,X2x,X2y,X2z,X3a,X3b,X3c,X3d,X3e,X3f,X3g,X4a,X4b,X4c,X4d,X5a,X5aa,X5ab,X5ac,X5ad,X5ae,X5af,X5ag,X5ah,X5b,X5c,X5d,X5f,X5g,X5h,X5i,X5j,X5k,X5l,X5m,X5n,X5o,X5p,X5q,X5r,X5s,X5t,X5u,X5v,X5w,X5x,X5y,X5z,X6a,X6b,X6c,X6d,X6e,X6f,X6g,X6h,X6i,X6j,X6k,X6l,X8a,X8b,X8c,X8d,X8e,X8f,X8g,X8h,X8i,X8j,X8k,X8l,X8m,X8n,X8o,X8p,X8q,X8r,X8s,X8t,X8u,X8v,X8w,X8x,X8y,row_sum,LPC1,LPC2,LPC3,LPC4,LPC5,LPC6,LPC7,LPC8,LPC9,LPC10,LPC11,LPC12,LPC13,LPC14,LPC15,LPC16,LPC17,LPC18,LPC19,LPC20,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59,PC60,PC61,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72,PC73,PC74,PC75,PC76,PC77,PC78,PC79,PC80,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90,PC91,PC92,PC93,PC94,hclusters,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,ID
0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0

In [177]:
print train_feat.shape
print test_feat.shape

(4208, 708)
(4209, 707)


In [181]:
train.shape

(4209, 378)

In [182]:
len(y_train)

4208

In [184]:
dtrain = xgb.DMatrix(train_feat.drop(['ID'], axis=1), y_train)
dtest = xgb.DMatrix(test_feat)


In [185]:
cv = xgb.cv(xgb_params, dtrain, 2000,\
            nfold=5L, feval = rsquare, maximize=True,early_stopping_rounds=40,verbose_eval=20)

[0]	train-r2:0.005492+9.14702e-05	test-r2:0.0050622+0.000184131
[20]	train-r2:0.105689+0.00142179	test-r2:0.101995+0.00148326
[40]	train-r2:0.189816+0.002505	test-r2:0.182786+0.0027532
[60]	train-r2:0.260418+0.00338731	test-r2:0.250141+0.0042863
[80]	train-r2:0.319801+0.00412665	test-r2:0.306278+0.0057212
[100]	train-r2:0.36976+0.00484764	test-r2:0.353268+0.00694433
[120]	train-r2:0.411864+0.0054304	test-r2:0.392462+0.00817672
[140]	train-r2:0.447335+0.00589052	test-r2:0.425222+0.00934149
[160]	train-r2:0.477256+0.0063494	test-r2:0.452463+0.0105615
[180]	train-r2:0.502615+0.00665272	test-r2:0.475125+0.0117961
[200]	train-r2:0.524183+0.00689776	test-r2:0.493922+0.012945
[220]	train-r2:0.54253+0.00709015	test-r2:0.509717+0.0142586
[240]	train-r2:0.558202+0.0072546	test-r2:0.522811+0.0153934
[260]	train-r2:0.57159+0.00738961	test-r2:0.533648+0.0165716
[280]	train-r2:0.583082+0.00747846	test-r2:0.542707+0.0177107
[300]	train-r2:0.593033+0.00756787	test-r2:0.550191+0.0186899
[320]	train-r2:

In [187]:
num_boost_rounds = 720

In [188]:
model_one = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

In [189]:
y_pred = model_one.predict(dtest)

In [204]:
y_pred[1:10]

array([  93.56087494,   79.79574585,   79.15461731,  111.9609375 ,
         92.96018982,  111.59954071,   94.09571838,  114.54527283,
         94.22277069], dtype=float32)

In [205]:
sub_xg = pd.DataFrame()
sub_xg['ID'] = test['ID']
sub_xg['y'] = y_pred
sub_xg.to_csv("20thJune/single_xgboost_python.csv",index=False) #0.544496

In [190]:
'''Train the stacked models then predict the test data'''

stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)),
    LassoLarsCV()

)

In [196]:
usable_columns = list(set(train_feat.columns) - set(['ID']))

In [197]:
finaltrainset = train_feat[usable_columns].values
finaltestset = test_feat.values

In [198]:
stacked_pipeline.fit(finaltrainset, y_train)
results = stacked_pipeline.predict(finaltestset)

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 17 iterations, i.e. alpha=1.470e-02, with an active set of 15 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 24 iterations, alpha=1.406e-02, previous alpha=1.380e-02, with an active set of 21 regressors.
  ConvergenceWarning)
/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 25 iterations, i.e. alpha=1.102e-02, with an active set of 23 regressors, and the smallest cholesky piv

In [199]:
print(r2_score(y_train,stacked_pipeline.predict(finaltrainset)*0.2855 + model_one.predict(dtrain)*0.7145))


0.654831414618


In [200]:
'''Average the preditionon test data  of both models then save it on a csv file'''

sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['y'] = y_pred*0.75 + results*0.25

In [202]:
sub.describe()

,ID,y
count,4209.000000,4209.000000
mean,4211.039202,103.876410
std,2423.078926,8.607016
min,1.000000,81.641678
25%,2115.000000,97.665457
50%,4202.000000,102.835701
75%,6310.000000,111.018563
max,8416.000000,129.381441


In [203]:
sub.to_csv('20thJune/stacked-models_two.csv', index=False) #0.371375